In [2]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
try:
    import empyrical as emp
except:
    emp = None
import tushare as ts
import time
import random
# from concurrent.futures import ProcessPoolExecutor

from common.log import *
from common.config import Config
from spider.spider_nasdaq import Spider_nasdaq
from spider.spider_coinmarketcap import Spider_coinmarketcap

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts

CONF = Config('../conf/secret.yaml').data[0]
ts_token = CONF['TUSHARE']['TOKEN']
ts.set_token(ts_token)
pro = ts.pro_api()

CONF = Config().data[0]
MONGODB = CONF['MONGODB']
NASDAQ = CONF['NASDAQ']
CRYPTOCURRENCY = CONF['CRYPTOCURRENCY']
CRYPTOCURRENCY = list(CRYPTOCURRENCY.keys())
NASDAQ100 = CONF['NASDAQ100']
HS300 = list(ts.get_hs300s()['code'])
ZZ500 = list(ts.get_zz500s()['code'])
SZ50 = list(ts.get_sz50s()['code'])
HSALL = list(set(ts.get_stock_basics().index) - set(['601162', '002940', '002939', '300674', '002941']))
HSALL = HSALL + ['000866', '600001', '000956']

BENCHMARK = '399300'
# TARGET = ['399300']
TARGET = HS300
# BENCHMARK = '000905'
# TARGET = ZZ500
# BENCHMARK = '000016'
# TARGET = SZ50
# TARGET = HSALL

# BENCHMARK = 'NDX'
# TARGET = NASDAQ100

# BENCHMARK = 'BITCOIN'
# TARGET = CRYPTOCURRENCY
# BENCHMARK = 'EOS'
# TARGET = ['EOS']

ALL_TARGET = TARGET[:]

pd
len(TARGET)
# TARGET

d:\python36\lib\site-packages\tushare\stock\classifying.py:266: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  ct.PAGES['sz50b']), parse_cols=[0, 4, 5])


<module 'pandas' from 'd:\\python36\\lib\\site-packages\\pandas\\__init__.py'>

300

In [2]:
SZ50_20121212 = ['600000', '600010', '600015', '600016', '600019', '600028',
 '600030', '600031', '600036', '600048', '600050', '600058',
 '600068', '600089', '600104', '600111', '600188', '600348',
 '600362', '600383', '600489', '600519', '600547', '600585',
 '600837', '600900', '601006', '601088', '601111', '601118',
 '601166', '601168', '601169', '601288', '601299', '601318',
 '601328', '601398', '601600', '601601', '601628', '601668',
 '601699', '601766', '601818', '601857', '601898', '601899',
 '601958', '601989']

SZ50_20151204 = ['600000', '600010', '600015', '600016', '600018', '600028',
'600030', '600036', '600048', '600050', '600111', '600104',
'600109', '600111', '600150', '600256', '600406', '600518',
'600519', '600583', '600585', '600637', '600690', '600837',
'600887', '600893', '600958', '600999', '601006', '601088',
'601166', '601169', '601186', '601288', '601318', '601328',
'601390', '601398', '601601', '601628', '601668', '601688',
'601766', '601800', '601818', '601857', '601901', '601988',
'601989', '601998']

SZ50_20160325 = ['600000', '600010', '600015', '600016', '600018', '600028',
'600030', '600036', '600048', '600050', '600104', '600109',
'600111', '600150', '600518', '600519', '600585', '600637',
'600795', '600837', '600887', '600893', '600958', '600999',
'601006', '601088', '601166', '601169', '601186', '601211',
'601288', '601318', '601328', '601336', '601390', '601398',
'601601', '601628', '601668', '601669', '601688', '601766',
'601800', '601818', '601857', '601901', '601985', '601988',
'601989', '601998']

SZ50_20161212 = ['600000', '600016', '600028', '600029', '600030', '600036',
'600048', '600050', '600100', '600104', '600109', '600111',
'600485', '600518', '600519', '600547', '600637', '600837',
'600887', '600893', '600958', '600999', '601006', '601088',
'601166', '601169', '601186', '601198', '601211', '601288',
'601318', '601328', '601336', '601377', '601390', '601398',
'601601', '601628', '601668', '601688', '601766', '601788',
'601800', '601818', '601857', '601901', '601985', '601988',
'601989', '601998']

SZ50_20171213 = ['600000', '600016', '600019', '600028', '600029', '600030',
'600036', '600048', '600050', '600104', '600111', '600309',
'600340', '600518', '600519', '600547', '600606', '600837',
'600887', '600919', '600958', '600999', '601006', '601088',
'601166', '601169', '601186', '601211', '601229', '601288',
'601318', '601328', '601336', '601390', '601398', '601601',
'601628', '601668', '601669', '601688', '601766', '601800',
'601818', '601857', '601878', '601881', '601985', '601988',
'601989', '603993']

FULL_SZ50 = SZ50_20121212 + SZ50_20151204 + SZ50_20160325 + SZ50_20161212 + SZ50_20171213 + list(ts.get_sz50s()['code'])
FULL_SZ50 = list(set(FULL_SZ50))

# TARGET = FULL_SZ50
# ALL_TARGET = TARGET[:]

d:\python3\lib\site-packages\tushare\stock\classifying.py:266: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  ct.PAGES['sz50b']), parse_cols=[0, 4, 5])


In [3]:
%%time

IS_DOWNLOAD_DATA = False
spider_n = Spider_nasdaq()
spider_c = Spider_coinmarketcap()
timeframe = '10y'
ignore = True

for symbol in TARGET + [BENCHMARK]:
    if not IS_DOWNLOAD_DATA:
        break
        
    print(symbol)

    # 沪深300指数
    if symbol in ['399300', '000905', '000016']:
        df = ts.get_k_data(code=symbol, index=True, start='2005-01-01', end=time.strftime('%Y-%m-%d'))
        datafile = '../database/market/%s.csv' % symbol
        df.to_csv(datafile, index=False, encoding='utf-8')
        
    # A股
    elif symbol in HS300 or symbol in ZZ500 or symbol in HSALL or symbol in FULL_SZ50:
        df = ts.get_k_data(code=symbol, ktype='D', autype='qfq', index=False, start='2005-01-01', end=time.strftime('%Y-%m-%d'))
        datafile = '../database/market/%s.csv' % symbol
        df.to_csv(datafile, index=False, encoding='utf-8')
        
    # NASDAQ
    elif symbol in NASDAQ100:
        datafile = None
        while datafile is None:
            try:
                datafile = spider_n.get_stock_data(symbol, timeframe)
            except:
                time.sleep(5)
    
    # CRYPTOCURRENCY
    elif symbol in CRYPTOCURRENCY:
        datafile = spider_c.get_coin_data(symbol, start_date='20100101', end_date=time.strftime('%Y%m%d'))
        
#     print(datafile)
    time.sleep(1)

Wall time: 0 ns


In [28]:
full_df.to_csv('../database/HS300IDX_2007-2009.csv', index=False, encoding='utf-8')

In [30]:
flag = 0
full_df = None
for today in pd.period_range(start='2015-01-01', end='2019-01-01', freq='D'):
    if flag == 0:
        start_date = today.strftime('%Y%m%d')
        flag = 1
    elif flag == 1:
        end_date = today.strftime('%Y%m%d')
        df = pro.index_weight(index_code='399300.SZ', start_date=start_date, end_date=end_date)
        if full_df is not None:
            full_df = pd.concat([full_df, df], ignore_index=True)
        else:
            full_df = df.copy()
        if len(df) != 0:
            today , len(df), len(full_df)
            pass
        flag = 0
        time.sleep(1)

len(full_df)
full_df.to_csv('../database/HS300IDX_2015-2018.csv', index=False, encoding='utf-8')
full_df



# start_date = '20050101'
# end_date = '20050201'
# pro.index_weight(index_code='399300.SZ', start_date=start_date, end_date=end_date)

(Period('2015-01-30', 'D'), 300, 300)

(Period('2015-02-27', 'D'), 300, 600)

(Period('2015-03-31', 'D'), 300, 900)

(Period('2015-04-30', 'D'), 300, 1200)

(Period('2015-05-30', 'D'), 300, 1500)

...

(Period('2018-09-03', 'D'), 300, 24300)

(Period('2018-09-29', 'D'), 300, 24600)

(Period('2018-10-09', 'D'), 300, 24900)

24900

,index_code,con_code,trade_date,weight
0,399300.SZ,600690.SH,20150130,0.3700
1,399300.SZ,600642.SH,20150130,0.2000
2,399300.SZ,600395.SH,20150130,0.0700
3,399300.SZ,000895.SZ,20150130,0.2600
4,399300.SZ,601601.SH,20150130,1.2800
5,399300.SZ,600549.SH,20150130,0.1300
6,399300.SZ,002236.SZ,20150130,0.1800
7,399300.SZ,600718.SH,20150130,0.1900
8,399300.SZ,000768.SZ,20150130,0.4000
9,399300.SZ,000060.SZ,20150130,0.1800
